Importing libraries

In [1143]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import SimpleImputer

Importing the dataset and picking relevant data

In [1144]:
raw_data = pd.read_csv('loan-data.csv',",")

X_features = pd.DataFrame(raw_data.iloc[:,0:11].values, columns = ['id','issue_date','loan_amnt','loan_status','funded_amnt','term','int_rate','installment','grade','sub_grade','verification_status'])
Y_dependable = pd.DataFrame(raw_data.iloc[:,12:14].values, columns = ['addr_state','total_paymnt'])

C:\Users\Slims\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


Check to see if missing data from the dataset

In [1145]:
X_missing_values = pd.isnull(X_features).sum()
Y_missing_values = pd.isnull(Y_dependable).sum()
print(X_missing_values, Y_missing_values)

id                       0
issue_date             500
loan_amnt              500
loan_status            500
funded_amnt            500
term                   500
int_rate               500
installment            501
grade                  515
sub_grade              514
verification_status    500
dtype: int64 addr_state      500
total_paymnt    500
dtype: int64


Since all issues dates are for the year 2022 it can be replaced

In [1146]:
change_month = pd.DataFrame(X_features["issue_date"])

change_month['month'] = change_month["issue_date"].str.split('/').str[0]
change_month['day'] = change_month["issue_date"].str.split('/').str[1]

change_month['day'] = change_month['day'].astype(float)


imputer = SimpleImputer(missing_values=np.nan, strategy='mean',verbose=0)
imputer.fit(change_month.iloc[:, 1:3])
change_month.iloc[:,1:3] = imputer.transform(change_month.iloc[:, 1:3]).round(0)
change_month['month'] = change_month['month'].astype(int).astype("string")
change_month['day'] = change_month['day'].astype(int).astype("string")
change_month['issue_date'] = change_month['month'] + "-" + change_month['day']
X_features['issue_date'] = change_month['issue_date']

Fix loan amount

In [1147]:
loan_amount = pd.DataFrame(X_features["loan_amnt"]).astype(float)
funded_amount = pd.DataFrame(X_features["funded_amnt"]).astype(float)
loan_amount["loan_amnt"] = loan_amount["loan_amnt"].fillna(funded_amount["funded_amnt"])

X_features["loan_amnt"] = loan_amount["loan_amnt"]
X_features = X_features.drop(["funded_amnt"], axis=1) #funded amount is same as loan amoutn so it is dropped

Fix loan status

In [1148]:
status_dict = {"Current": 1 ,"Fully Paid" : 1,"Issued" :1,
               "Charged Off":0, "Default":0, "In Grace Period":0,"Late (16-30 days)":0,
                      "Late (31-120 days)":0, np.nan : 0}

In [1149]:
loan_status_change =  pd.DataFrame(X_features["loan_status"])
X_features["loan_status"]  = loan_status_change["loan_status"].map(status_dict) 


Fix term

In [1159]:
term_encoding = {" 36 months": 1, " 60 months": 0, np.nan: 0}
X_features['term'] = X_features['term'].map(term_encoding)
print(X_features)

            id issue_date  loan_amnt  loan_status  term  int_rate  \
0     48010226       5-15    35000.0            1   NaN     13.33   
1     57693261       7-15    30000.0            1   NaN      0.89   
2     59432726       9-15    15000.0            1   NaN      0.53   
3     53222800       7-15     9600.0            1   NaN      0.89   
4     57803010       8-15     8075.0            1   NaN     19.19   
...        ...        ...        ...          ...   ...       ...   
9995  42974433       3-15    18000.0            1   NaN      0.18   
9996  45424654       4-15    11300.0            1   NaN      0.99   
9997  50415990       6-15    10000.0            1   NaN      0.89   
9998  46154151       4-15    10000.0            1   NaN     16.55   
9999  66055249      12-15    10000.0            1   NaN      0.26   

      installment  sub_grade  verification_status  
0         1184.86         13                    1  
1          938.57          5                    1  
2          494.

   Fix Interest rate

In [1151]:
interest_rate = pd.DataFrame(X_features["int_rate"])
imputer = SimpleImputer(missing_values=np.nan, strategy='mean',verbose=0)
imputer.fit(interest_rate.iloc[:])
X_features["int_rate"] = imputer.transform(interest_rate.iloc[:])

Fix installment

In [1152]:
installment = pd.DataFrame(X_features["installment"])
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') 
imputer.fit(installment.iloc[:])
X_features["installment"] = imputer.transform(installment.iloc[:]).round(2)


Fix grades and subgrades

In [1153]:
grade_dict = {"A":5,"B":10,"C":15,"D":20,"E":25,"F":30,"G":35,"A1":1,"A2":2,"A3":3,"A4":4,"A5":5,
              "B1":6,"B2":7,"B3":8,"B4":9,"B5":10,"C1":11,"C2":12,"C3":13,"C4":14,"C5":15,
              "D1":16,"D2":17,"D3":18,"D4":19,"D5":20,"E1":21,"E2":22,"E3":23,"E4":24,"E5":25,
              "F1":26,"F2":27,"F3":28,"F4":29,"F5":30,"G1":31,"G2":32,"G3":33,"G4":34,"G5":35, np.nan:36}
new_subgrades = pd.DataFrame(X_features["grade"])
new_subgrades["sub_grade"] = pd.DataFrame(X_features["sub_grade"])
new_subgrades["sub_grade"] = new_subgrades["sub_grade"].fillna(new_subgrades["grade"])
X_features = X_features.drop(["grade"], axis=1)
new_subgrades["sub_grade"] = new_subgrades["sub_grade"].map(grade_dict)
X_features["sub_grade"]  =new_subgrades["sub_grade"].astype(int) 


Fix verification status

In [1154]:
verification_dict = {"Verified": 1, "Source Verified": 1, "Not Verified":0, np.nan: 0}

X_features["verification_status"]  = X_features["verification_status"].map(verification_dict) 

Fix address state

In [1155]:
regions_dict = { "PA": "East", "NY":"East","VT": "East", "NH":"East","ME": "East",
               "MA":"East","RI": "East", "CT":"East", "NJ":"East","WA": "West", 
               "OR":"West","CA": "West", "NV":"West","ID": "West", "MT":"West","WY": 
               "West", "UT":"West", "AZ":"West","NM": "West", "CO":"West", "AK":"West",
               "ND":"MidWest","SD": "MidWest", "NE":"MidWest","KS": "MidWest", "MN":"MidWest",
               "IA": "MidWest", "MO":"MidWest", "IL":"MidWest","WI": "MidWest", "IN":"MidWest", 
               "MI":"MidWest","OH":"MidWest","OK":"South", "TX":"South", "AR":"South", "LA":"South", 
               "MS":"South", "TN":"South", "KY":"South", "AL":"South", "FL":"South", 
               "GA":"South", "SC":"South", "NC":"South", "VA":"South", "WV":"South", "DC":"South", 
               "MD":"South", "DE":"South", np.nan: 0}
           
Y_dependable["addr_state"]  = Y_dependable["addr_state"].map(regions_dict) 
address_state = pd.DataFrame(Y_dependable["addr_state"])
imputer = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value=0)
imputer.fit(address_state.iloc[:])
Y_dependable["addr_state"] = imputer.transform(address_state.iloc[:])


Fix Total Payment

In [1156]:
total_payment = pd.DataFrame(Y_dependable["total_paymnt"])
imputer.fit(total_payment.iloc[:])
Y_dependable["total_paymnt"] = imputer.transform(total_payment.iloc[:])

Convert Currency to EURO

In [1157]:
xchangrate_data = pd.read_csv('EUR-USD.csv',",")

xchangrate_data = pd.DataFrame(xchangrate_data.iloc[:,3].values, columns = ["Close"])

#X_features['quantity'] = X_features['quantity'].apply(lambda x: x*xchangrate_data.iloc)

C:\Users\Slims\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


CHECK for missing data on the final dataframe 

In [1158]:
credit_check_dataframe = pd.concat([X_features, Y_dependable], axis=1)
credit_check_dataframe = credit_check_dataframe.sort_values(by=['issue_date'])
print(final_dataframe)
missing_values = pd.isnull(credit_check_dataframe).sum()
print(missing_values)
credit_check_dataframe.to_csv('Credit_check.csv')

     issue_date  loan_amnt  loan_status  term   int_rate  installment  \
2291       1-15    20000.0            0     1  13.660000       680.26   
591        1-15    35000.0            1     1  19.990000      1300.55   
592        1-15     6400.0            1     1   0.490000       196.13   
2425       1-15    16000.0            1     1   0.490000       512.46   
8293       1-15     8500.0            1     0  14.310000       291.80   
...         ...        ...          ...   ...        ...          ...   
830        9-15    21000.0            1     0   0.170000       437.67   
824        9-15     9600.0            1     1  17.570000       345.00   
821        9-15    28000.0            1     1  15.610000       979.02   
9006       9-15    12000.0            1     1   7.524344       361.38   
5269       9-15     8375.0            1     1  18.250000       303.83   

      sub_grade  verification_status addr_state total_paymnt  
2291         13                    1      South            0